**Installing the required libraries**

In [1]:
!pip install azure-ai-ml azure-identity

**Importing the dependencies**

In [1]:
import os
import json
import datetime

from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential

**Configure the resource details**

In [2]:
# laoding the configuration file
config_file_path = "config.json"

# Read JSON data into a dictionary
config_data = json.load(open(config_file_path))

subscription_id = config_data["subscription_id"]
resource_group = config_data["resource_group"]  
workspace = config_data["workspace"]
location = config_data["location"]

In [3]:
print(resource_group)
print(workspace)
print(location)

deploy-plant-dis-cnn-rg
deploy-lant-dis-cnn-ws
West US


**Resource_Group and Workspace**

**NOTE: Create ResourceGroup and AzureMachineLeanringWorkspace with these names**

(You can also try creating the resource group and workspace from python azure sdk)

In [4]:
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [6]:
# Define an endpoint name
endpoint_name = "endpt-potato-dis" 

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="This is an online endpoint for predicting Plant diseases",
    auth_mode="aml_token"
)

In [7]:
print(endpoint_name)

endpt-potato-dis


**Configuring the environment**

**NOTE: Check the quotas to make sure that instance type is available in the chosen region**

vm sku list: https://learn.microsoft.com/en-us/azure/machine-learning/reference-managed-online-endpoints-vm-sku-list?view=azureml-api-2

In [8]:
model = Model(path="trained_model/plant_disease_model_v1.keras")
env = Environment(
    conda_file="conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="onlinescoring", scoring_script="score.py"
    ),
    instance_type="Standard_DS1_v2",
    instance_count=1,
)


**Creating the endpoint**

In [9]:
%%time
create_endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

CPU times: total: 2.28 s
Wall time: 1min 48s


**Deploying the endpoint**

In [10]:
%%time
from azure.core.exceptions import HttpResponseError
try:
    deploy_endpoint = ml_client.online_deployments.begin_create_or_update(blue_deployment).result()
except HttpResponseError as ex:
    print("Erro ao implantar o endpoint:", ex)

Instance type Standard_DS1_v2 may be too small for compute resources. Minimum recommended compute SKU is Standard_DS3_v2 for general purpose endpoints. Learn more about SKUs here: https://learn.microsoft.com/en-us/azure/machine-learning/referencemanaged-online-endpoints-vm-sku-list
Check: endpoint endpt-potato-dis exists


........................................................................CPU times: total: 3.19 s
Wall time: 7min 24s


**You can print create_endpoint and deploy_endpoint and cjheck teh details**

In [11]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
add_traffic = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

**Get details of the endpoint**

In [12]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

{'blue': 100}
https://endpt-potato-dis.westus.inference.ml.azure.com/score


**Testing the endpoint**

In [13]:
import base64
import json

# Read the image
with open('0a8a68ee-f587-4dea-beec-79d02e7d3fa4___RS_Early.B 8461.JPG', 'rb') as image_file:
    image_data = image_file.read()

# Convert image to base64
image_base64 = base64.b64encode(image_data).decode('utf-8')

json_request = json.dumps({'data': image_base64})

# Save this JSON as a file
json_file_path = 'sample-request.json'  # Specify your desired path for the JSON file
with open(json_file_path, 'w') as json_file:
    json_file.write(json_request)


# test the blue deployment with some sample data
response = ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name="blue",
    request_file="sample-request.json",
)

In [14]:
print(response)

"{\"Predicted Class Name\": \"Potato___Early_blight\"}"


In [126]:
# Convert the string to a dictionary
prediction = json.loads(response)

# Now dict_data is a Python dictionary
print(prediction)

{"Predicted Class Name": "Potato___Early_blight"}


get AML token: https://learn.microsoft.com/en-us/rest/api/azureml/online-endpoints/get-token?view=rest-azureml-2023-10-01&tabs=HTTP

In [15]:
print(resource_group)
print(workspace)
print(endpoint_name)

deploy-plant-dis-cnn-rg
deploy-lant-dis-cnn-ws
endpt-potato-dis


Getting the logs

In [ ]:
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=endpoint_name, lines=50
)

Deleting the endpoint

In [130]:
# deleting the endpoint
ml_client.online_endpoints.begin_delete(name=endpoint_name)